**What is Pandas?**

Pandas is a python library with a focus on efficient exploration and manipulation of data.
At the core of this library lies the `Dataframe` which is essentially an multidimensional array, that can contain all possible kinds of data, including missing values, in a comprehensive, explicitly labeld form. 
While we are used to the numpy array comprised of numerical values, which are great for statistical 
and numerical operations, a Dataframe allows additionally the implementation of common spreadsheet operations 
that go beyond simple element-wise boradcasting, like grouping by variables, transformations or pivoting.

This notebook is meant to be an introduction to the common operations necessary in the neurosciences. It provides a hopefully concise starting point, but is in it's scope not exhaustive.

- adapted from the [Python data science handbook](https://github.com/jakevdp/PythonDataScienceHandbook)
please consider supporting the author by pruchasing the [book](http://shop.oreilly.com/product/0636920034919.do)

See Documentation:

- the pandas [cookbook](https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook): a extensive library of useful examples and tricks

- [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) including all pandas internal functions and examples of how to implement them


## Pandas Objects

First, let's introduce the main objects of the Pandas library we'll be confronted with.



In [ ]:
# standard imports at the start of your script
import numpy as np
import pandas as pd

## Pandas Series Object

a one-dimensional array of indexed data

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0]*2)

display(data)
print('values: ', data.values)
print('indices: ', data.index)

Like with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [ ]:
display(data[1])
display(data[0:4])
display(data[1::2])

``Series`` has an *explicitly defined* index associated with the values.

This explicit index definition gives the ``Series`` object additional capabilities. For example, the index need not be an integer, but can consist of values of any desired type.
For example, if we wish, we can use strings as an index:

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
display(data)

print('-'*40)
display(data.index)  # show index

print('-'*40)
display(data.index[0:2]) # slice-selecting

print('-'*40)
display(type(data.index[0:2]))

print('-'*40)
display(data['d'])  # specifiy value by index

print('-'*40)
print('explicitly selecting subset by index')
display(data[data.index[0:3]])
values = data[data.index[0:3]]

display(values[0])  # extract value
display(values['a'])  # extract value

###  Indexing: `explicit` vs `implicit`indexing

In [ ]:
data = pd.Series(['a', 'b', 'c','d','e'], index=[1, 3, 5, 7, 9])
# explicit index when indexing
display(data[1])

# implicit index when slicing
display(data[1:3])


display(data.loc[1:5])  # explicit
display(data.iloc[1:5])  # implicit

### Series as a specialized dictionary

constructing a ``Series`` object directly from a Python dictionary:

In [ ]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

By default, a ``Series`` will be created where the index is drawn from the sorted keys.
From here, typical dictionary-style item access can be performed:

In [ ]:
display(population['California'])

population['California':'Illinois'] # series also supports array-style operations such as slicing, unlike dict

### Main event: the Pandas `DataFrame`
two-dimensional array with both flexible row indices and flexible column names.
* think of it as an sequence of series, where the initial index of the series defines the row index (the y-axis) and the "title" of the series defines it's specific column name (x-axis)

In [ ]:
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}

area = pd.Series(area_dict)  # define a variable containig a series

data = pd.DataFrame({'population': population,
                       'area': area}) 
data['density'] = data['population'] / data['area']
display(data)

print('-'*40)

print('indices: ')
display(data.index)  

print('-'*40)

print('columns: ')
display(data.columns)

#### What happens if series do not share the same indices?

In [ ]:
conflicting_data = pd.Series(list(range(6)), index=['a','b','c','d','e','f'])

conflicting_states = pd.DataFrame({'population': population,
                       'area': area,
                       'dict':conflicting_data}) 
conflicting_states

In [ ]:
display(pd.DataFrame([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}]))


#### additional ways to build a dataframe

In [ ]:
# from a set of lists
a = list(i for i in range(3))
b = list(range(3))

df = {'a': a, 'b': b} 
df = pd.DataFrame(data)
display(df)

# from a 2d numpy array
pd.DataFrame(np.random.rand(3, 2),
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

#### accessing ``Series`` of a Dataframe

In [ ]:
display(data['area'])  # by column name

display(data.area)  # attribute style

data.area is data['area']

#### accessing rows and columns of a Dataframe by array-style indexing
- ``loc`` & ``iloc``

In [ ]:
display(data.iloc[:3, :2])  # access the first three rows and the first two columns by implicit indexing

display(data.loc[:'Illinois', :'population'])  # loc indexer allows the usage of explicit index and column name

data.loc[data.density > 100, ['population', 'density']]  # index by masking

these conventions can also be used to assign values to specific rows/columns/cells

In [ ]:
data_copy = data.copy()  # make a copy of your df so you don't overwrite essential data

data_copy['age_range'] = np.nan  # creates a new column with none-type values
display(data_copy)

data_copy.iloc[0, 2] = None  # set specific cell to none-type
display(data_copy)

data_copy.loc[data_copy.density > 100, ['density']] = '100'
display(data_copy)

## Manipulating data in Pandas

### Detecting null values
Pandas data structures have two useful methods for detecting null data: ``isnull()`` and ``notnull()``.
Either one will return a Boolean mask over the data. For example:

In [ ]:
display(data.isna())  

display(data_copy.isna())

### Check present `Datatypes` of your Dataframe

In [ ]:
display(data.dtypes)

display(data_copy.dtypes)

print('-'*40)

display(type(data_copy['age_range'][0])) #check type of specific value


You can see that our missing values `nan` are treated as float64. Therefore we can still preform numerical operations on this column

In [ ]:
print('sum')
display(data_copy.sum())  # sum up values of individual columns

print('-'*40)
print('mean')
display(data_copy.mean())  # means of indivdual columns

print('-'*40)

sort by values with [`.sort_values()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)

In [ ]:
display(data_copy.sort_values('population'))  # sort by values in column
display(data_copy.sort_values('population', ascending=False))

return basic descriptive staistics with the [`.describe()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) function

In [ ]:
display(data_copy.describe()) # return basic descriptive staistics

display(data_copy['density'].describe())  # return basic descriptive staistics of single column

### Exercise
The density colum seems to contain data of the type `"object"`. What does this mean and how would you use what you have learned today to locate possible problems in this colum? 
Try also to use the `.describe`function on the other columns. How would you interpret the different output of `area` and `density`?

In [ ]:
#solution

#inspecting single cell
display(type(data_copy['density'][0]))
display(type(data_copy['density'][3]))

# use a for loop to iterate over the column values
for i in range(len(data_copy['density'])):
    print(data_copy['density'][i])
    print(type(data_copy['density'][i]))

### Dealing with missing data

In [ ]:
df = data_copy.copy()  # copy your df to prevent data loss

In [ ]:
df.dropna()  # drop row containing missing values

That's no good. The 'age_range' column unfortunately contains only missing values, therefore we are left with no data at all.

In [ ]:
df = data_copy.copy()
df.dropna(axis='columns')  # drop columns containing missing values

Better, but we still lost the 'density' column, because of a single missing value.
How would you drop a single row of our DataFrame?

In [ ]:
# extract sub_df manually by slicing
display(df)
df = data_copy[1:]  # get rid of california
display(df)

df.dropna(axis='columns')

extract sub_df manually by using the implicit index

In [ ]:
df = data_copy.drop(data_copy.index[0])
display(df)

df.dropna(axis='columns')

drop column by explicit index

In [ ]:
df = data_copy.copy()
df =df.drop(['California'])
display(df)

df = df.drop('age_range', axis=1)  # axis = 1 denotes that we are refering to columns
df

When calling df.describe() we still see that columns containing different `dtypes` are missing

In [ ]:
df.describe()

The `astype`function allows us to make sensible transformations between `dtypes`. This becomes especially apparent, when working with comma-separated files (.csv files) which may contain `strings` instead of `floats`, when following the german convention of separating integer and fractional part of a number with a comma instead of a decimal point (e.g. 8,00 vs 8.00).

So let's recast our values to the `flaot64` type and into `string` type to illustrate another aspect of the `.describe()` function

In [ ]:
df = data_copy.copy()
df = df.astype('float64')
display(df.describe())

# or cast to string for dealing with categorical data
df = data_copy.copy()
df_text = df.astype('str')
display(df_text.describe())

Let's also transform our missing values to 0s making it easier to deal with single missing values.

In [ ]:
df = data_copy.copy()
df = df.fillna(0)  # Replace all NaN elements with 0s
display(df)

df = data_copy.copy()  # # Replace all NaN elements with 1s
df = df.fillna(1)
df
# display(df.describe())

# Combining Datasets: Concat and Append

### Simple concatenation with ``pd.concat``

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

Dataframes can be simply split by indexing

In [ ]:
df = pd.DataFrame(np.random.randn(10, 4))
piece_1, piece_2, piece_3 = df[:3], df[3:7], df[7:] 
display(piece_1)
display(piece_2)
display(piece_3)

#### The `pd.concat()` funtion

In [ ]:
df = pd.concat([piece_1,piece_2, piece_3])  # combine frames
display(df)

### Excercise

Be careful with overlapping indices. The `ignore_index` parameter merely assigns a new index.
Try setting ignore_index to true and compare rows `0 - 2` with the the rows `6 - 8`. 

In [ ]:
piece_4 = piece_3  # make a copy 
display(pd.concat([piece_3,piece_1, piece_4], axis=0, join="inner", ignore_index = False))

Let's create a little more complex DataFrame

In [ ]:
df = pd.DataFrame([['bird', 'polly'], ['monkey', 'jane'], ['dog', 'pavlov'], ['duck', 'konrad'], ['cat', 'erwin']],
                    columns=['animal', 'name'])

df2 = pd.DataFrame([['e', 5, 'bird'], ['b', 2, 'monkey'],['c', 3, 'cat'], ['d', 4, 'dog'], ['a', 1, 'duck']],
                           columns=['letter', 'number', 'animal'])
display(df)

display(df2)

In [ ]:
display(pd.concat([df, df2], axis=0, join="outer", ignore_index = True))

### Exercise
This doesn't look right.
Play around with the parameters `axis`, `join`, `ignore_index` to fix the DataFrame. See -> [concat()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat)

In [ ]:
# solution
display(pd.concat([df, df2], axis=1, join="inner"))

In [ ]:
display(df)
display(df2.columns)

#### The `merge()`method

In [ ]:
df.columns
df2.columns
pd.merge(df, df2,on='animal')

#### The ``append()`` method

Instead of calling `pd.concat([df, df2])`, you can simply call `df.append(df2)`, when appropiate. 

In [ ]:
df = pd.DataFrame([['a', 1], ['b', 2]], columns=['strings', 'integers'])
df2 = pd.DataFrame([['c', 3], ['d', 4]], columns=['strings', 'integers'])

display(df.append(df2, ignore_index=True))

## GroupBy: Split, Apply, Combine

- The *split* step involves breaking up and grouping a ``DataFrame`` depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
- The *combine* step merges the results of these operations into an output array.

Let's load the Planets dataset, from the [Seaborn package](http://seaborn.pydata.org/) adapted from the [kaggle open exoplanet catalogue ](https://www.kaggle.com/mrisdal/open-exoplanet-catalogue.) to have a real world example.

In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

In [ ]:
planets.head()

The most basic split-apply-combine operation can be computed with the ``groupby()`` method of ``DataFrame``s, passing the name of the desired key column:

In [ ]:
planets.groupby('mass')

you can apply virtually any common Pandas or NumPy aggregation function, as well as virtually any valid ``DataFrame`` operation

In [ ]:
planets.groupby('method').sum()

In [ ]:
planets.groupby('year').describe().T  # descriptive stats for discovered planets by year

You can also select a particular `series`

In [ ]:
display(planets.groupby('method')['distance'].median())  # median distance from Sun in Parsec (3.26 Light-years = 1 Parsec)

In [ ]:
print(' Top methods per year')
display(planets.groupby('year')['method'].describe())  # which methods where mainly used in a given year

# compare table to graph
# from the seaborn visualisation tutorial
with sns.axes_style('white'):
    g = sns.catplot("year", data=planets, aspect=4.0, kind='count',
                       hue='method', order=range(1994, 2015))
    g.set_ylabels('Number of Planets Discovered')

#### Iteration over groups

The ``GroupBy`` object supports direct iteration over the groups, returning each group as a ``Series`` or ``DataFrame``:

In [ ]:
for method, group in planets.groupby('method'):
    print("{0:30s} shape={1}  type={2}".format(method, group.shape, type(group)))
display(type(df))

In [ ]:
for method, group in planets.groupby(['method']):
#     display(method)
    display(group.head())

### Excercise

Use a for-loop like the one above to group the planets DataFrame by year, drop all columns containing missing data and print the result for every iteration.

In [ ]:
# solution 
for year, group in planets.groupby(['year']):
    display(year)
    group = group.dropna(axis='columns')
    display(group.head().T)

#### Or use the build-in `get_group` function.

In [ ]:
imaging_planets = planets.groupby(['method']).get_group('Imaging')
imaging_planets.head()

In [ ]:
planets.groupby(['method', 'year']).get_group(('Radial Velocity', 2014))

### Loading and saving Data
Most data that one deals with is either saved in the csv or tsv format.
Let's import some relevant [Data](https://www.kaggle.com/nickhould/craft-cans) 


Generally use the [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function to load and the [`to_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) function to save data

In [ ]:
# try excluding the index_col and the sep parameter
df = pd.read_csv('data/beers.csv', index_col=0)
df.head()
df['brewery_id'] = df['brewery_id'].astype('category')
df.head()

Appy some sensible action like dropping some unnecessary columns and sorting by alochol content.

In [ ]:
df_cleaned = df.drop(['ibu', 'id', 'style', 'brewery_id'], axis=1)
df_cleaned = df_cleaned.sort_values('abv', ascending=False)
df_cleaned = df_cleaned[0:5]
display(df_cleaned)

Save your cleaned Data again.

In [ ]:
df_cleaned.to_csv('data/shoppin_list.tsv', sep='\t', index=False)

## Reshaping Data

Pandas offers multiple functions to reshape Data. In general these offer the same basic functionaliyt as the groupby function, but excel in efficiency and readability.

#### The [`pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table) function.

Allows for hierarchical and multiindexing

In [ ]:
pivoted = pd.pivot_table(df,index=["style","name"])
pivoted.T

In [ ]:
# extract some aspect by indices
Altbier = pivoted['id']['Altbier']
Altbier

Apply some function using the newly created hierarchical index

In [ ]:
def style_info(beer):
    print('-'*50)
    print(beer)
    print('median alochol content (0 = no alcohol, 1 = pure alcohol) : ' + str(pivoted['abv'][beer].median()))
    print('median bitterness (International bittering units): ' + str(pivoted['ibu'][beer].median()))
    print('median size of beer in ounces: ' + str(pivoted['ounces'][beer].median()))
    print('-'*50)
    
    
style_info('Abbey Single Ale')

style_info('Altbier')

style_info('American Black Ale')
style_info('American Blonde Ale')

In [ ]:
pivoted_2 = pd.pivot_table(df,index=["name","style"])
display(pivoted_2.T)

def beer_info(beer):
    print('-'*50)
    print(beer)
    print('median alochol content (0 = no alcohol, 1 = pure alcohol) : ' + str(pivoted_2['abv'][beer].median()))
    print('median bitterness (International bittering units): ' + str(pivoted_2['ibu'][beer].median()))
    print('median size of beer in ounces: ' + str(pivoted_2['ounces'][beer].median()))
    print('-'*50)
    
beer_info('10 Degrees of Separation')

#### The [`pivot`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html) function
- llows more flexibility, but isn't as intuive to use.

Let's extract a list of products

In [ ]:
# looks pretty useless atm
style = df.pivot(index='id', columns='style', values='name')
display(style.head())

brews = df.pivot(index='id', columns='brewery_id', values='name')
display(brews.head())


Lets drop all missing values and print a list of AIPA brands

In [ ]:
print('AIPA brands')
aipa = style['American IPA'].dropna()
display(aipa[0:20])

print('--'*50)
print('All brews of brewery with ID "60"')

# lets drop all missing values and print a list of all brews of brewery "60"
brews_60 = brews[60].dropna()
display(brews_60)

### The [`.stack()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html#pandas.DataFrame.stack) and [`unstack()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html#pandas.DataFrame.unstack) functions

#### Stack from columns to index  -> wide to long format

In [ ]:
df_stacked = df.stack()
display(df_stacked[0:20])

#### Stack from index to columns -> long to wide format

In [ ]:
df_stacked.unstack().head()

### The [`.melt()`]() function

unpivots a DataFrame from wide format to long format


- id_vars[tuple, list, or ndarray, optional] : Column(s) to use as identifier variables.
- value_vars[tuple, list, or ndarray, optional]: Column(s) to unpivot. If not specified, uses all columns that are not set as id_vars.
- var_name[scalar]: Name to use for the ‘variable’ column. If None it uses frame.columns.name or ‘variable’.
- value_name[scalar, default ‘value’]: Name to use for the ‘value’ column.

In [ ]:
melted = df.melt(id_vars=['name', 'style'], value_vars =['ounces', 'abv'])
melted

## Futher resources:

- [10 miutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html): a short introduction for new users

- the pandas [tutorial](http://pandas.pydata.org/pandas-docs/version/0.15/tutorials.html): a more detailed course divied into specific lessons
    
- [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) by Wes McKinney (original creator of Pandas)

- [Pandas on PyVideo](http://pyvideo.org/search?q=pandas): featured tutorials from Pandas developers and power users. The PyCon tutorials in particular tend to be given by very well-vetted presenters.
